In [8]:
%pip install pandas
%pip install numpy 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [10]:
df_apple = pd.read_csv("C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados/data_apple_tv.csv")

In [11]:
df_apple

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2381047.0,NaN
1,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1249470.0,NaN
2,Citizen Kane,movie,"Drama, Mystery",1941.0,tt0033467,8.3,481447.0,NaN
3,Metropolis,movie,"Drama, Sci-Fi",1927.0,tt0017136,8.3,194630.0,NaN
4,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1251651.0,NaN
...,...,...,...,...,...,...,...,...
18724,NaN,tv,NaN,2025.0,NaN,NaN,NaN,NaN
18725,Wild Sky,tv,Documentary,2025.0,tt36615591,NaN,NaN,NaN
18726,The Sun,tv,Documentary,2024.0,tt33397097,NaN,NaN,NaN
18727,NaN,tv,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_apple.describe()

,releaseYear,imdbAverageRating,imdbNumVotes
count,18692.000000,16768.000000,1.676800e+04
mean,2006.962497,6.369060,2.498266e+04
std,18.591890,1.165882,9.898266e+04
min,1902.000000,1.300000,5.000000e+00
25%,2001.000000,5.600000,1.990000e+02
50%,2014.000000,6.500000,1.212500e+03
75%,2020.000000,7.200000,7.878750e+03
max,2026.000000,10.000000,2.381047e+06


In [13]:
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18729 entries, 0 to 18728
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               18135 non-null  object 
 1   type                18729 non-null  object 
 2   genres              18037 non-null  object 
 3   releaseYear         18692 non-null  float64
 4   imdbId              17163 non-null  object 
 5   imdbAverageRating   16768 non-null  float64
 6   imdbNumVotes        16768 non-null  float64
 7   availableCountries  57 non-null     object 
dtypes: float64(3), object(5)
memory usage: 1.1+ MB


In [14]:
df_apple.duplicated().sum()

np.int64(398)

In [15]:
# Verificar colunas com valores duplicados
for col in df_apple.columns:
    num_dups = df_apple.duplicated(subset=[col]).sum()
    if num_dups > 0:
        print(f"Coluna '{col}' possui {num_dups} valores duplicados.")

Coluna 'title' possui 1286 valores duplicados.
Coluna 'type' possui 18727 valores duplicados.
Coluna 'genres' possui 17910 valores duplicados.
Coluna 'releaseYear' possui 18621 valores duplicados.
Coluna 'imdbId' possui 1571 valores duplicados.
Coluna 'imdbAverageRating' possui 18646 valores duplicados.
Coluna 'imdbNumVotes' possui 10703 valores duplicados.
Coluna 'availableCountries' possui 18714 valores duplicados.


In [16]:
df_apple.drop_duplicates()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2381047.0,NaN
1,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1249470.0,NaN
2,Citizen Kane,movie,"Drama, Mystery",1941.0,tt0033467,8.3,481447.0,NaN
3,Metropolis,movie,"Drama, Sci-Fi",1927.0,tt0017136,8.3,194630.0,NaN
4,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1251651.0,NaN
...,...,...,...,...,...,...,...,...
18721,The Motherhood,tv,Reality-TV,2025.0,tt34729564,4.6,28.0,NaN
18722,Das Boot - Die komplette TV-Serie,tv,"Drama, War",1987.0,tt30970892,8.9,243.0,NaN
18725,Wild Sky,tv,Documentary,2025.0,tt36615591,NaN,NaN,NaN
18726,The Sun,tv,Documentary,2024.0,tt33397097,NaN,NaN,NaN


In [17]:
df_apple.isnull().sum()

title                   594
type                      0
genres                  692
releaseYear              37
imdbId                 1566
imdbAverageRating      1961
imdbNumVotes           1961
availableCountries    18672
dtype: int64

In [18]:
df_apple.dropna(subset=['title'])

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2381047.0,NaN
1,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1249470.0,NaN
2,Citizen Kane,movie,"Drama, Mystery",1941.0,tt0033467,8.3,481447.0,NaN
3,Metropolis,movie,"Drama, Sci-Fi",1927.0,tt0017136,8.3,194630.0,NaN
4,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1251651.0,NaN
...,...,...,...,...,...,...,...,...
18721,The Motherhood,tv,Reality-TV,2025.0,tt34729564,4.6,28.0,NaN
18722,Das Boot - Die komplette TV-Serie,tv,"Drama, War",1987.0,tt30970892,8.9,243.0,NaN
18725,Wild Sky,tv,Documentary,2025.0,tt36615591,NaN,NaN,NaN
18726,The Sun,tv,Documentary,2024.0,tt33397097,NaN,NaN,NaN


In [19]:
#preencher os NaN de genres com "Não informado"
df_apple['genres'] = df_apple['genres'].fillna('Não informado')

In [20]:
# os NaN em imdbId e imdbAverageRating podem ser um problema se for analisar a avaliação dos filmes, então preciso ver se o NaN em um coincide com NaN na outra

# Investigar a situação dos NaN
print("Registros sem imdbId:", df_apple['imdbId'].isna().sum())
print("Registros sem rating:", df_apple['imdbAverageRating'].isna().sum())

# Ver se NaN em imdbId implica em NaN em rating
mask_sem_id = df_apple['imdbId'].isna()
print("Dos que não têm ID, quantos também não têm rating:", 
      df_apple[mask_sem_id]['imdbAverageRating'].isna().sum())

Registros sem imdbId: 1566
Registros sem rating: 1961
Dos que não têm ID, quantos também não têm rating: 1566


In [21]:
# Investigar os casos com ID mas sem avaliação
tem_id_sem_rating = df_apple['imdbId'].notna() & df_apple['imdbAverageRating'].isna()
print("Registros com ID mas sem avaliação:", tem_id_sem_rating.sum())

# Ver alguns exemplos desses casos
print("\nExemplos de conteúdo com ID mas sem avaliação:")
exemplos = df_apple[tem_id_sem_rating][['title', 'imdbId', 'releaseYear']].head()
print(exemplos)

Registros com ID mas sem avaliação: 395

Exemplos de conteúdo com ID mas sem avaliação:
                                   title      imdbId  releaseYear
3802                         Engal Kural  tt10127906       1985.0
4077                       Episode #1.26   tt5544420       1974.0
4083                      Yo soy porteño   tt5544454       1982.0
4090  Thou Shalt Not Worship False Idols   tt5544492       2016.0
4091        Episode dated 11 August 2014   tt5544502       2014.0


In [22]:
#preencher os NaN de imdbid com "Não informado"
df_apple['imdbId'] = df_apple['imdbId'].fillna('Não informado')

In [23]:
def impute_ratings_by_genre_decade(df_apple):
    """
    Imputa valores NaN de imdbAverageRating usando mediana por gênero primário + década
    
    Args:
        df: DataFrame com colunas 'genres', 'releaseYear', 'imdbAverageRating'
    
    Returns:
        DataFrame com ratings imputados e colunas adicionais para rastreamento
    """
    # Extrair gênero primário (primeiro da lista)
    df_apple['primaryGenre'] = df_apple['genres'].fillna('Unknown').str.split(',').str[0].str.strip()

    # Extrair década
    df_apple['decade'] = (df_apple['releaseYear'] // 10) * 10

    # Flag para indicar se tinha rating original
    df_apple['hasOriginalRating'] = df_apple['imdbAverageRating'].notna()

    # Calcular medianas por gênero + década (apenas filmes COM rating)
    films_with_rating = df_apple[df_apple['hasOriginalRating']]
    
    medians_genre_decade = (films_with_rating.groupby(['primaryGenre', 'decade'])['imdbAverageRating'].agg(['median', 'count']).reset_index())

    # Calcular medianas só por gênero (fallback)
    medians_genre = (films_with_rating.groupby('primaryGenre')['imdbAverageRating'].median().reset_index())
    
    # Mediana global (último recurso)
    global_median = films_with_rating['imdbAverageRating'].median()

    # Função aninhada para imputar um filme específico
    def impute_single_film(row):
        if row['hasOriginalRating']:
            return pd.Series({
                'imdbAverageRating': row['imdbAverageRating'],
                'ratingImputed': False,
                'imputationSource': 'original'
            })
        
        # Tentar gênero + década primeiro
        genre_decade_match = medians_genre_decade[
            (medians_genre_decade['primaryGenre'] == row['primaryGenre']) & 
            (medians_genre_decade['decade'] == row['decade'])
        ]
        
        if not genre_decade_match.empty:
            median_val = genre_decade_match.iloc[0]['median']
            count = genre_decade_match.iloc[0]['count']
            return pd.Series({
                'imdbAverageRating': median_val,
                'ratingImputed': True,
                'imputationSource': f"{row['primaryGenre']} {int(row['decade'])}s ({count} filmes)"
            })
        
        # Fallback: só gênero
        genre_match = medians_genre[medians_genre['primaryGenre'] == row['primaryGenre']]
        
        if not genre_match.empty:
            median_val = genre_match.iloc[0]['imdbAverageRating']
            return pd.Series({
                'imdbAverageRating': median_val,
                'ratingImputed': True,
                'imputationSource': f"{row['primaryGenre']} geral"
            })
        
        # Último recurso: mediana global
        return pd.Series({
            'imdbAverageRating': global_median,
            'ratingImputed': True,
            'imputationSource': "mediana global"
        })

    # Aplicar imputação
    imputation_results = df_apple.apply(impute_single_film, axis=1)

    # Combinar resultados
    df_apple['imdbAverageRating'] = imputation_results['imdbAverageRating']
    df_apple['ratingImputed'] = imputation_results['ratingImputed']
    df_apple['imputationSource'] = imputation_results['imputationSource']

    return df_apple, medians_genre_decade, global_median

In [24]:
# APLICAR A IMPUTAÇÃO
df_imputed, medians_table, global_median = impute_ratings_by_genre_decade(df_apple)

In [25]:
imputed_count = df_imputed['ratingImputed'].sum()
print(f"Valores imputados: {imputed_count}")

Valores imputados: 1961


In [26]:
# Mostrar alguns exemplos organizados
for i, (_, filme) in enumerate(df_imputed.head(8).iterrows()):
    print(f"{i+1:2d}. \"{filme['title']}\" ({int(filme['releaseYear'])})")
    print(f"    Gênero: {filme['primaryGenre']}")
    print(f"    Avaliação imputada: {filme['imdbAverageRating']:.1f}")
    print(f"    Fonte: {filme['imputationSource']}")
    print()

 1. "Forrest Gump" (1994)
    Gênero: Drama
    Avaliação imputada: 8.8
    Fonte: original

 2. "American Beauty" (1999)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original

 3. "Citizen Kane" (1941)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original

 4. "Metropolis" (1927)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original

 5. "Kill Bill: Vol. 1" (2003)
    Gênero: Action
    Avaliação imputada: 8.2
    Fonte: original

 6. "Unforgiven" (1992)
    Gênero: Drama
    Avaliação imputada: 8.2
    Fonte: original

 7. "Eternal Sunshine of the Spotless Mind" (2004)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original

 8. "Amores Perros" (2000)
    Gênero: Drama
    Avaliação imputada: 8.0
    Fonte: original



In [27]:
# Axis 1 para colunas, axis 0 para linhas
df_apple = df_apple.drop(['availableCountries'], axis=1)

In [28]:
df_apple.isnull().sum()

title                 594
type                    0
genres                  0
releaseYear            37
imdbId                  0
imdbAverageRating       0
imdbNumVotes         1961
primaryGenre            0
decade                 37
hasOriginalRating       0
ratingImputed           0
imputationSource        0
dtype: int64

In [29]:
# Ver quais são os 4 filmes problemáticos
nulls_year = df_apple[df_apple['releaseYear'].isna()]
print(nulls_year[['title', 'type', 'genres', 'imdbId', 'imdbAverageRating']])

                               title   type                        genres  \
10037                         Natudu  movie             Romance, Thriller   
14278  KillShot: or Murder Man and M  movie         Action, Comedy, Crime   
14689                            NaN     tv                 Não informado   
14934                            NaN     tv                   Documentary   
14958                            NaN     tv                         Drama   
14974                            NaN     tv                 Não informado   
14991                            NaN     tv                 Não informado   
15009                            NaN     tv                 Não informado   
15163                            NaN     tv                 Não informado   
15224                            NaN     tv                 Não informado   
15301                            NaN     tv                        Comedy   
15310                            NaN     tv                         Drama   

In [30]:
# Dropando...
df_apple.dropna(subset=['releaseYear'], inplace=True)

In [31]:
#### só depois de remover todos os NaN é que posso converter o releaseYear para int
df_apple['releaseYear'] = df_apple['releaseYear'].astype(int)

In [32]:
# ==========================================
# LIMPEZA FINAL - REMOVER BAGAGEM TÉCNICA
# ==========================================

# 1. TRATAR imdbNumVotes para filmes imputados
print("🔧 TRATANDO imdbNumVotes PARA FILMES IMPUTADOS")
print("=" * 50)

# Para filmes com rating imputado, vamos usar a mediana de votos do mesmo gênero/década
films_with_votes = df_apple[df_apple['imdbNumVotes'].notna()]

# Calcular mediana de votos por gênero primário
median_votes_by_genre = (films_with_votes
                        .groupby('primaryGenre')['imdbNumVotes']
                        .median()
                        .to_dict())

# Mediana global como fallback
global_median_votes = films_with_votes['imdbNumVotes'].median()

# Função para imputar votos
def impute_votes(row):
    if pd.notna(row['imdbNumVotes']):
        return row['imdbNumVotes']
    
    # Se tem rating imputado, usar mediana do gênero
    if row['ratingImputed']:
        genre_median = median_votes_by_genre.get(row['primaryGenre'])
        if genre_median:
            return genre_median
        else:
            return global_median_votes
    
    return row['imdbNumVotes']



🔧 TRATANDO imdbNumVotes PARA FILMES IMPUTADOS


In [33]:
# Aplicar imputação de votos
df_apple['imdbNumVotes'] = df_apple.apply(impute_votes, axis=1)


In [34]:
df_apple['imdbNumVotes'] = df_apple['imdbNumVotes'].astype(int)
# Verificar se a conversão foi bem-sucedida
print(df_apple.dtypes)

title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
primaryGenre          object
decade               float64
hasOriginalRating       bool
ratingImputed           bool
imputationSource      object
dtype: object


In [35]:
# CRIAR DATASET LIMPO (apenas colunas essenciais) e removendo as colunas criadas para o tratamento
print(f"\n📂 CRIANDO DATASET LIMPO")
print("=" * 30)

colunas_essenciais = [
    'title', 
    'type', 
    'genres', 
    'releaseYear', 
    'imdbId', 
    'imdbAverageRating', 
    'imdbNumVotes'
]

df_apple_clean = df_apple[colunas_essenciais].copy()

print(f"Forma final do dataset: {df_apple_clean.shape}")
print(f"Colunas finais: {list(df_apple_clean.columns)}")




📂 CRIANDO DATASET LIMPO
Forma final do dataset: (18692, 7)
Colunas finais: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes']


In [36]:
# VERIFICAÇÃO FINAL
print(f"\n✅ VERIFICAÇÃO FINAL")
print("=" * 35)

print("Valores nulos por coluna:")
print(df_apple_clean.isnull().sum())

print(f"\nTipos de dados:")
print(df_apple_clean.dtypes)


✅ VERIFICAÇÃO FINAL
Valores nulos por coluna:
title                561
type                   0
genres                 0
releaseYear            0
imdbId                 0
imdbAverageRating      0
imdbNumVotes           0
dtype: int64

Tipos de dados:
title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
dtype: object


# ========Exploração Tipo e Gênero =============

In [37]:
tipos_unicos = df_apple_clean['type'].unique()
print("Tipos únicos na coluna 'type':", tipos_unicos)

Tipos únicos na coluna 'type': ['movie' 'tv']


In [38]:
# Contando quantas produções existem de cada tipo
contagem_tipos = df_apple_clean['type'].value_counts()
print(f"\nDistribuição por tipo:")
for tipo, quantidade in contagem_tipos.items():
    percentual = (quantidade / len(df_apple)) * 100
    print(f"{tipo}: {quantidade} produções ({percentual:.1f}%)")


Distribuição por tipo:
movie: 14363 produções (76.8%)
tv: 4329 produções (23.2%)


In [39]:
# EXPLORANDO OS PADRÕES DOS GENRES
print("\nPADRÕES E ESTRUTURA DOS GENRES")
# Amostras aleatórias para entender o formato
print("Exemplos de como os genres aparecem:")
sample_genres = df_apple_clean['genres'].dropna().sample(10).values
for i, genre in enumerate(sample_genres, 1):
    print(f"{i:2d}. {genre}")


PADRÕES E ESTRUTURA DOS GENRES
Exemplos de como os genres aparecem:
 1. Drama, Short
 2. Drama, Romance
 3. Crime, Drama, Mystery
 4. Comedy, Drama, Romance
 5. Drama, War
 6. Drama
 7. Documentary, History
 8. Comedy
 9. Reality-TV
10. Documentary, History


In [40]:
# Função para extrair genres individuais
def extract_individual_genres(df_apple_clean):
    """Extrai todos os genres individuais de uma coluna que pode conter múltiplos genres"""
    all_genres = []
    
    for genres_str in df_apple_clean['genres'].dropna():
        # Remove espaços e divide por vírgulas (assumindo que é o separador mais comum)
        if pd.notna(genres_str):
            # Tratamento flexível para diferentes separadores
            genres_list = re.split(r'[,;|]', str(genres_str))
            # Limpa e adiciona à lista
            clean_genres = [genre.strip() for genre in genres_list if genre.strip()]
            all_genres.extend(clean_genres)
    
    return all_genres

In [41]:
# Extraindo todos os genres individuais
individual_genres = extract_individual_genres(df_apple_clean)

In [42]:
# Contando frequencias
genre_counts = Counter(individual_genres)
print(f"Total de genres únicos encontrados: {len(genre_counts)}")
print(f"Total de ocorrências de genres: {sum(genre_counts.values())}")

Total de genres únicos encontrados: 38
Total de ocorrências de genres: 37763


In [43]:
# TOP GENRES MAIS POPULARES
print("\n TOP 20 GENRES MAIS POPULARES")
top_genres = genre_counts.most_common(20)
for i, (genre, count) in enumerate(top_genres, 1):
    percentage = (count / sum(genre_counts.values())) * 100
    print(f"{i:2d}. {genre:<25} | {count:>6} ocorrências ({percentage:>5.2f}%)")



 TOP 20 GENRES MAIS POPULARES
 1. Drama                     |   8485 ocorrências (22.47%)
 2. Comedy                    |   4749 ocorrências (12.58%)
 3. Romance                   |   2746 ocorrências ( 7.27%)
 4. Crime                     |   2699 ocorrências ( 7.15%)
 5. Documentary               |   2511 ocorrências ( 6.65%)
 6. Thriller                  |   2229 ocorrências ( 5.90%)
 7. Action                    |   2140 ocorrências ( 5.67%)
 8. Horror                    |   1792 ocorrências ( 4.75%)
 9. Mystery                   |   1627 ocorrências ( 4.31%)
10. Adventure                 |   1261 ocorrências ( 3.34%)
11. Family                    |   1183 ocorrências ( 3.13%)
12. History                   |    782 ocorrências ( 2.07%)
13. Biography                 |    746 ocorrências ( 1.98%)
14. Não informado             |    667 ocorrências ( 1.77%)
15. Fantasy                   |    649 ocorrências ( 1.72%)
16. Animation                 |    560 ocorrências ( 1.48%)
17. Sci-F

In [44]:
df_apple_clean.to_csv("df_apple_clean.csv", index=False)